In [2]:
from sgl_models import *
import os
import joblib
import random
import numpy as np
torch.backends.cudnn.enabled=False
from sklearn.metrics import roc_auc_score
import time


torch.backends.cudnn.deterministic = True
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sparsity = 0.1
bag_size = 10
X_test = joblib.load('/data2/user/MNIST/X_test_%d_%d_%d'%((int(sparsity*100), bag_size, 250)))
y_test = joblib.load('/data2/user/MNIST/y_test_%d_%d_%d'%((int(sparsity*100), bag_size, 250)))
actual_y_test = joblib.load('/data2/user/MNIST/actual_y_test_%d_%d_%d'%((int(sparsity*100), bag_size, 250)))

test_dg = CIFAR10Dataset('test', {'features':torch.tensor(X_test), 'labels':torch.tensor(y_test), 'labels2':torch.tensor(actual_y_test)})
test_loader = DataLoader(test_dg,batch_size = 1,shuffle = False)

/home/user/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### SGL_NOPE

In [6]:
cM = 40
cE = 100
cL = 500

cD = 128
    
model = SGLModel(ftEx, cM, cE, cL, cD, PE = False).to(device)
bag_preds = []
bag_ys = []
start_epoch = time.time()
for curridx, x_i, y_i, actual_y_i in test_loader:
    x_i = x_i.transpose(0, 1)
    y_i = y_i.to(device)
    
    # positive pair, with encoding
    inst_pred = model(x_i.to(device))
    bag_prediction = lin_soft(inst_pred.sigmoid())

    bag_preds.extend(bag_prediction.detach().cpu().numpy())
    bag_ys.extend(y_i.detach().cpu().numpy())

    del x_i
    del bag_prediction
end_epoch = time.time()

bag_preds = np.array(bag_preds)
bag_ys = np.array(bag_ys)  
print(end_epoch - start_epoch)

1.115872859954834


### SGL_PE

In [7]:
cM = 40
cE = 100
cL = 500

cD = 128
cLR = 1e-4
cWD = 1e-7

model = SGLModel(ftEx, cM, cE, cL, cD, PE = True).to(device)

bag_preds = []
bag_ys = []
start_epoch = time.time()
for curridx, x_i, y_i, actual_y_i in test_loader:
    x_i = x_i.transpose(0, 1)
    y_i = y_i.to(device)
    
    # positive pair, with encoding
    inst_pred = model(x_i.to(device))
    bag_prediction = lin_soft(inst_pred.sigmoid())

    bag_preds.extend(bag_prediction.detach().cpu().numpy())
    bag_ys.extend(y_i.detach().cpu().numpy())

    del x_i
    del bag_prediction
end_epoch = time.time()

bag_preds = np.array(bag_preds)
bag_ys = np.array(bag_ys)  
print(end_epoch - start_epoch)

1.4989159107208252
